## ЗАДАНИЕ

#### Вам необходимо проверить: 
##### - есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
##### - различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.
###### Проверку производите с использованием инструментов, изученных в ходе выполнения кейса.

In [1]:
import pandas as pd

#### 1. Загружаем данные (выбираем только тех пользователи, что зарегистрировались в 2018 году).

In [2]:
# Загружаем данные
events = pd.read_csv( \
    'file:///C:/Users/Marat/Desktop/Python/SkillFactory-python/Data/7_4_Events.csv', sep=',')

purchase = pd.read_csv( \
    'file:///C:/Users/Marat/Desktop/Python/SkillFactory-python/Data/purchase.csv', sep=',')

In [3]:
# Делаем копии
events_df = events.copy()
purchase_df = purchase.copy()

In [4]:
# Выбираем пользователей, зарегистрировавшихся в 2018 и добавляем в список registration_list
registration_list = events_df[
    (events_df['start_time'] >= '2018-01-01') \
        & (events_df['start_time'] < '2019-01-01') \
            & (events_df['event_type'] == 'registration')
]['user_id'].to_list()        

In [5]:
# таблицы events_df и purchase_df ограничиваем списком registration_list (выбираем 2018г.)
events_df = events_df[events_df['user_id'].isin(registration_list)] 
purchase_df = purchase_df[purchase_df['user_id'].isin(registration_list)]   

#### 2. Произвести предобработку данных: исследование, преобразование типов, где это необходимо.

In [6]:
# Выделяем событие оплаты
purchase_df['event_type'] = 'purchase'

In [7]:
# Переименовываем id
events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})

In [8]:
# Преобразуем даты в DataTime
events_df['start_time'] = pd.to_datetime(events_df['start_time'])
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])

In [9]:
# Смотрим какие поля можно преобразовать в category
print(events_df.nunique())
print(purchase_df.nunique())
print(events_df['event_type'].unique())
print(events_df['selected_level'].unique())
print(purchase_df['event_type'].unique())

event_id          66959
event_type            5
selected_level        3
start_time        66809
tutorial_id       18050
user_id           19926
dtype: int64
purchase_id       1600
user_id           1600
event_datetime    1600
amount               7
event_type           1
dtype: int64
['registration' 'tutorial_start' 'tutorial_finish' 'level_choice'
 'pack_choice']
[nan 'medium' 'hard' 'easy']
['purchase']


In [10]:
# Преобразуем event_type и selected_level в category
events_df['event_type'] = events_df['event_type'].astype('category')
events_df['selected_level'] = events_df['selected_level'].astype('category')
purchase_df['event_type'] = purchase_df['event_type'].astype('category')

In [11]:
# Объединяем таблицы
total_events_df = pd.concat([events_df,purchase_df],sort=False)

In [12]:
# Преобразуем event_type в category (так как добавился новый тип)
total_events_df['event_type'] = total_events_df['event_type'].astype('category')

In [13]:
# Удаляем индекс и сортируем по start_time
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

#### 3. Создаем группы пользователей в зависимости от выбранной сложности.

In [14]:
# Смотрим какие группы будем создавать
total_events_df['selected_level'].unique()

[NaN, 'medium', 'hard', 'easy']
Categories (3, object): ['easy', 'hard', 'medium']

In [15]:
# Создаем 3 таблицы по каждую группу easy_level_df, medium_level_df и hard_level_df
easy_level_df = total_events_df[total_events_df['selected_level'] == 'easy']
medium_level_df = total_events_df[total_events_df['selected_level'] == 'medium']
hard_level_df = total_events_df[total_events_df['selected_level'] == 'hard']

In [16]:
# Проверяем что user_id для каждой таблицы определены однозначно
print(easy_level_df['user_id'].value_counts().mean())
print(medium_level_df['user_id'].value_counts().mean())
print(hard_level_df['user_id'].value_counts().mean())

1.0
1.0
1.0


In [17]:
# Оставим только те данные, которые нужны для наших вычислений
easy_level_df = easy_level_df[["user_id", "start_time"]].rename(columns={"start_time": "easy_level_time"})
medium_level_df = medium_level_df[["user_id", "start_time"]].rename(columns={"start_time": "medium_level_time"})
hard_level_df = hard_level_df[["user_id", "start_time"]].rename(columns={"start_time": "hard_level_time"})

In [18]:
# Для каждой группы создаем списки с уникальными user_id
easy_level_list = easy_level_df["user_id"].unique()
medium_level_list = medium_level_df["user_id"].unique()
hard_level_list = hard_level_df["user_id"].unique()

In [19]:
# Создаем для каждой группы таблицу с оплатами
purchase_df_1 = purchase_df[purchase_df["user_id"].isin(easy_level_list)]
purchase_df_2 = purchase_df[purchase_df["user_id"].isin(medium_level_list)]
purchase_df_3 = purchase_df[purchase_df["user_id"].isin(hard_level_list)]

In [20]:
# Оставим только те данные, которые нужны для наших вычислений
purchase_df_1 = purchase_df_1[["user_id", "event_datetime"]].rename(columns={"event_datetime": "purchase_time"})
purchase_df_2 = purchase_df_2[["user_id", "event_datetime"]].rename(columns={"event_datetime": "purchase_time"})
purchase_df_3 = purchase_df_3[["user_id", "event_datetime"]].rename(columns={"event_datetime": "purchase_time"})

#### 4. Для каждой группы подсчитаем процент пользователей, оплативших пакеты вопросов.

In [21]:
# Вычисляем процент оплат для группы easy
percent_of_purchase_1 = purchase_df_1['user_id'].nunique() / len(easy_level_list)

print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших уровень easy): {:.2%}".format(
        percent_of_purchase_1
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших уровень easy): 7.72%


In [22]:
# Вычисляем процент оплат для группы medium
percent_of_purchase_2 = purchase_df_2['user_id'].nunique() / len(medium_level_list)

print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших уровень medium): {:.2%}".format(
        percent_of_purchase_2
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших уровень medium): 20.86%


In [23]:
# Вычисляем процент оплат для группы hard
percent_of_purchase_3 = purchase_df_3['user_id'].nunique() / len(hard_level_list)

print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших уровень hard): {:.2%}".format(
        percent_of_purchase_3
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших уровень hard): 35.39%


#### 5. Для каждой группы посчитаем среднее время между событиями оплаты и выбором уровня сложности.

In [24]:
# Мерджим таблицы уровней с таблицами оплат
merged_df_1 = easy_level_df.merge(purchase_df_1, on = "user_id", how = "inner")
merged_df_2 = medium_level_df.merge(purchase_df_2, on = "user_id", how = "inner")
merged_df_3 = hard_level_df.merge(purchase_df_3, on = "user_id", how = "inner")

In [25]:
# Добавляем поле timedelta - между событиями оплаты и выбором уровня сложности
merged_df_1["timedelta"] = (merged_df_1["purchase_time"] - merged_df_1["easy_level_time"])
merged_df_2["timedelta"] = (merged_df_2["purchase_time"] - merged_df_2["medium_level_time"])
merged_df_3["timedelta"] = (merged_df_3["purchase_time"] - merged_df_3["hard_level_time"])

In [26]:
# Среднее время между событиями оплаты и выбором уровня сложности easy
print(merged_df_1["timedelta"].mean())
print(merged_df_1["timedelta"].describe())

3 days 14:58:52.941798941
count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object


In [27]:
# Среднее время между событиями оплаты и выбором уровня сложности medium
print(merged_df_2["timedelta"].mean())
print(merged_df_2["timedelta"].describe())

3 days 23:14:13.165118679
count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%                5 days 16:07:19
max               10 days 13:51:01
Name: timedelta, dtype: object


In [28]:
# Среднее время между событиями оплаты и выбором уровня сложности hard
print(merged_df_3["timedelta"].mean())
print(merged_df_3["timedelta"].describe())

3 days 07:20:41.420814479
count                          442
mean     3 days 07:20:41.420814479
std      1 days 21:43:52.953292605
min                0 days 03:26:45
25%         1 days 14:57:23.500000
50%         3 days 03:13:57.500000
75%         4 days 19:16:00.250000
max                8 days 01:18:13
Name: timedelta, dtype: object


#### 6. Выводы о различии в показателях между группами

##### Процент пользователей, которые оплатили пакет вопросов значительно увеличивается с увеличением сложности уровня.  Т.е. выбор уровня влияет на мотивации дальнейшего использования приложения.
##### Временные результаты можно использовать для оптимизации медианного времени т.к. квартиль значительно ниже

#### 7. Проверям, существует ли разница во времени между событиями регистрации и оплаты.

In [29]:
# Выделим отдельную таблицу registration_df, который будет содержать только события с event_type = registration
registration_df = total_events_df[total_events_df['event_type'] == 'registration']

In [30]:
# Создаем для каждой группы таблицу с регистрациями
registration_df_1 = registration_df[registration_df["user_id"].isin(easy_level_list)]
registration_df_2 = registration_df[registration_df["user_id"].isin(medium_level_list)]
registration_df_3 = registration_df[registration_df["user_id"].isin(hard_level_list)]

In [31]:
# Оставим только те данные, которые нужны для наших вычислений
registration_df_1 = registration_df_1[["user_id", "start_time"]].rename(columns={"start_time": "registration_time"})
registration_df_2 = registration_df_2[["user_id", "start_time"]].rename(columns={"start_time": "registration_time"})
registration_df_3 = registration_df_3[["user_id", "start_time"]].rename(columns={"start_time": "registration_time"})

In [32]:
# Мерджим таблицы регисраций с таблицами оплат
merged_df_4 = registration_df_1.merge(purchase_df_1, on = "user_id", how = "inner")
merged_df_5 = registration_df_2.merge(purchase_df_2, on = "user_id", how = "inner")
merged_df_6 = registration_df_3.merge(purchase_df_3, on = "user_id", how = "inner")

In [33]:
# Добавляем поле timedelta - между событиями оплаты и выбором уровня сложности
merged_df_4["timedelta"] = (merged_df_4["purchase_time"] - merged_df_4["registration_time"])
merged_df_5["timedelta"] = (merged_df_5["purchase_time"] - merged_df_5["registration_time"])
merged_df_6["timedelta"] = (merged_df_6["purchase_time"] - merged_df_6["registration_time"])

In [34]:
# Среднее время между событиями оплаты и регистрацией для пользователей, выбравших уровень сложности easy
print(merged_df_4["timedelta"].mean())
print(merged_df_4["timedelta"].describe())

3 days 22:10:23.211640211
count                          189
mean     3 days 22:10:23.211640211
std      2 days 07:14:41.062010764
min                0 days 04:36:58
25%                2 days 01:12:12
50%                3 days 11:00:23
75%                5 days 10:24:59
max               11 days 00:35:04
Name: timedelta, dtype: object


In [35]:
# Среднее время между событиями оплаты и регистрацией для пользователей, выбравших уровень сложности medium
print(merged_df_5["timedelta"].mean())
print(merged_df_5["timedelta"].describe())

4 days 06:12:06.576883384
count                          969
mean     4 days 06:12:06.576883384
std      2 days 06:25:57.480868026
min                0 days 08:39:24
25%                2 days 08:46:51
50%                4 days 03:35:26
75%                5 days 23:51:27
max               10 days 20:34:02
Name: timedelta, dtype: object


In [36]:
# Среднее время между событиями оплаты и регистрацией для пользователей, выбравших уровень сложности hard
print(merged_df_6["timedelta"].mean())
print(merged_df_6["timedelta"].describe())

3 days 14:55:19.257918552
count                          442
mean     3 days 14:55:19.257918552
std      1 days 22:22:52.441896774
min                0 days 09:41:39
25%         1 days 23:36:25.500000
50%         3 days 10:10:04.500000
75%         5 days 03:30:07.750000
max                8 days 14:21:29
Name: timedelta, dtype: object


##### при выборе уровня medium времени от регистрации до оплаты тратиться больше чем при выборе других уровней сложности. 